In [ ]:
import re
import nltk
import pandas as pd
import datetime
import numpy as np
from nltk.stem.porter import PorterStemmer
englishstemmer=nltk.stem.SnowballStemmer('english')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import sys
import os
from sklearn.pipeline import Pipeline
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.preprocessing.text import Tokenizer
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline
plt.style.use('ggplot')

def init():
    data = pd.read_csv('../Combined_News_DJIA.csv')
    train = data[data['Date'] < '2015-01-01']
    test = data[data['Date'] > '2014-12-31']
    testlines = []
    for row in range(0,len(train.index)):
        testlines.append(' '.join(str(x) for x in train.iloc[row,2:27]))
    return testlines

def setupVectorizer():
    basicvectorizer = CountVectorizer()
    basictrain = basicvectorizer.fit_transform(trainheadlines)
    print(basictrain.shape)
    return basictrain

def setupMLP():
    batch_size = 32
    nb_classes = 2
    advancedvectorizer = TfidfVectorizer( min_df=0.04, max_df=0.3, max_features = 200000, ngram_range = (2, 2))
    advancedtrain = advancedvectorizer.fit_transform(trainlines)
    testlines = []
    for row in range(0,len(test.index)):
        testlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
    advancedtest = advancedvectorizer.transform(testlines)
    print(advancedtrain.shape)
    return advancedtrain, advancedtest

def genData(advancedtrain, advancedtest):
    X_train = advancedtrain.toarray()
    X_test = advancedtest.toarray()
    print('X_train shape:', X_train.shape)
    print('X_test shape:', X_test.shape)
    y_train = np.array(train["Label"])
    y_test = np.array(test["Label"])
    Y_train = np_utils.to_categorical(y_train, nb_classes)
    Y_test = np_utils.to_categorical(y_test, nb_classes)
    scale = np.max(X_train)
    X_train /= scale
    X_test /= scale
    mean = np.mean(X_train)
    X_train -= mean
    X_test -= mean
    input_dim = X_train.shape[1]
    return input_dim
    
def createMLP(input_dim):
    model = Sequential()
    model.add(Dense(256, input_dim=input_dim))
    model.add(Activation('relu'))
    model.add(Dropout(0.4))
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dropout(0.4))
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))
    return model
    
if __name__ == "__main__":
    testlines = init()
    basictrain = setupVectorizer()
    advancedtrain, advancedtest = setupMLP()
    dims = genData(advancedtrain, advancedtest)
    model = createMLP(dims)
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc']))
    print("Training...")
    model.fit(X_train, Y_train, nb_epoch=2, batch_size=16, validation_split=0.15, show_accuracy=True)
    print("Generating test predictions...")
    preds14 = model.predict_classes(X_test, verbose=0)
    acc14 = accuracy_score(test["Label"], preds14)
    print('prediction accuracy: ', acc14)